In [1]:
import numpy as np
import numpy.linalg as lin
from typing import Tuple, List

In [2]:

class TimpaniData:
    def __init__(c_trans: float, alpha: float, b: float, b_3: float):
        self.c_trans = c_trans
        self.alpha = alpha
        self.w_0 = 0
        self.b = b # (paramètre de viscosité)
        self.b_3 = b_3 # (coefficient relié à la force due au rayonnement de la membrane)

"""[summary]
Implements « Synthèse sonore de sons de tiimbale, A. Ramdane, 1990
source: http://www.edpsciences.org/10.1051/jphyscol:19902201
Differences in notation :
- w is used as the name for the transversal displacement of the membrane.
- n is used as the number of discrete time points (and not N).
- The time iterator's name is k (and not n).
"""
class TimpaniSolver:
    
    def __init__(d: TimpaniData, dt, n: int):
        """[summary]
        Args:
            d (TimpaniData): [description]
            steps (Tuple[float]): [description]
            n (int): [description]
        """
        self.d = d
        self.dt = dt
        # dx and dy are set from dt, so that we have r_1**2 + r_2**2 = 1
        self.dx = self.dy = 2 / (self.dt * self.d.c_trans) ** 2
        self.n = n

        self.h = self.dx # ? not sure about what that's supposed to be, or the value.

        self.r_1 = self.d.c_trans * self.dt / self.dx
        self.r_2 = self.d.c_trans * self.dt / self.dy

        self.epsilon_1 = lambda i, j, k: np.sqrt(self.n ** 2 - j ** 2)/self.h - i/self.h
        self.epsilon_2 = lambda i, j, k: np.sqrt(self.n ** 2 - j ** 2)/self.h - i/self.h
    
    def compute_w(i, j, k):
        """[summary]

        Args:
            i ([type]): [description]
            j ([type]): [description]
            k ([type]): [description]
        """
        if k <= 0:
            # At $t_0$, the displacement is set everywhere on the membrane.
            return self.d.w_0
        region = self.__get_point_region((i, j))
        w = None
        if region == 0:
            return self.d_w_0
        else if region == 1:
            e_1 = self.epsilon(i, j, k)
            e_2 = self.epsilon(i, j, k)
            a = 2 * self.r_1**2 / (1 + e_1 * self.h)
            # pretty sure there is an error in the pdf of the article for this coefficient.
            # for now, I copied it as-is.
            b = - 2 * (self.r_1**2 (self.h / e_1 + h / e_2 - 1 )
            c = 2 * self.r_1**2 / (1 + e_2 * h)
            d = -1
            #
            # TODO: cache
            return a * compute_w(i-1, j, k-1) + b * compute_w(i, j, k-1) + c * compute_w(i, j-1, k-1) + d * compute_w(i, j, k-2)
        else:
            pass 
        return w


    def __get_point_region(self, p: Tuple[int]):
        """[summary]
        Get the region of the (i, j) point,
        can be 0: circular rim or outside
            1: neighbourhood of the circular rim on the inside
            2: inside the circle
        Args:
            n ([type]): [description]
            p (Tuple[int]): Point of indices (i, j) in the grid.
        """
        radius = self.n * 0.5
        norm = lin.norm(np.array(p, dtype=np.float64))
        diff = radius - norm
        if norm < radius:
            return 0
        elif norm >= radius - 1:
            return 1
        else:
            return 2 




SyntaxError: invalid syntax (3805963963.py, line 9)